In [ ]:
!rm -r sample_data

In [ ]:
!unzip /content/drive/MyDrive/Segmentation/dataSet/ph2dataset.zip

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam

In [ ]:
H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y


In [ ]:
# augmenting data
import cv2
from glob import glob
from albumentations import CenterCrop, RandomRotate90, HorizontalFlip, VerticalFlip
from tqdm import tqdm

def augment(images,masks,save_path,augment_data=True):
  H=256
  W=256
  for x,y in tqdm(zip(images,masks),total=len(images)):
    name=x.split('/')[-1].split(".")
    img_name=name[0]
    img_ext=name[1]

    name=x.split('/')[-1].split(".")
    mask_name=name[0]
    mask_ext=name[1]

    x=cv2.imread(x,cv2.IMREAD_COLOR)
    y=cv2.imread(y,cv2.IMREAD_COLOR)
    #print(x.shape)
    #print(y.shape)

    if augment_data==True:
      # Randome rotate
      aug=RandomRotate90(p=0.8)
      augmented=aug(image=x,mask=y)
      x2=augmented['image']
      y2=augmented['mask']
      # vertical Flip
      aug=VerticalFlip(p=0.6)
      augmented=aug(image=x,mask=y)
      x3=augmented['image']
      y3=augmented['mask']
      # Horizontal Flip Flip
      aug=HorizontalFlip(p=0.7)
      augmented=aug(image=x,mask=y)
      x4=augmented['image']
      y4=augmented['mask']
      #Save file
      save_images=[x,x2,x3,x4]
      save_masks=[y,y2,y3,y4]
    else:
      save_images=[x]
      save_masks=[y]
    idx=0
    for i,m in zip(save_images,save_masks):
      i=cv2.resize(i,(W,H))
      m=cv2.resize(m,(W,H))
      tmp_img_name=f"{img_name}_{idx}.{img_ext}"
      tmp_mask_name=f"{mask_name}_{idx}.{mask_ext}"
      image_path=os.path.join(save_path,"image",tmp_img_name)
      mask_path=os.path.join(save_path,"mask",tmp_mask_name)
      cv2.imwrite(image_path,i)
      cv2.imwrite(mask_path,m)
      idx+=1

In [ ]:
create_dir("augDataset")
create_dir("augDataset/image")
create_dir("augDataset/mask")

In [ ]:
path='/content/ph2_dataset'
images=sorted(glob(os.path.join(path,"trainx/*")))
masks=sorted(glob(os.path.join(path,"trainy/*")))
print(f"Orignal Image: {len(images)}")
print(f"Orignal Masks: {len(masks)}")

Orignal Image: 200
Orignal Masks: 200


In [ ]:
augment(images,masks,'/content/augDataset',True)

100%|██████████| 200/200 [00:04<00:00, 49.26it/s]


In [ ]:
# remove unwanted folders
!rm -r /content/ph2_dataset
!rm -r /content/trainx
!rm -r /content/trainy

In [ ]:
path='/content/augDataset'
images=sorted(glob(os.path.join(path,"image/*")))
masks=sorted(glob(os.path.join(path,"mask/*")))
print(f"Augmented Image: {len(images)}")
print(f"Augmented Masks: {len(masks)}")

Augmented Image: 800
Augmented Masks: 800


In [ ]:
def load_data(dataset_path, split=0.2):
    images = sorted(glob(os.path.join(dataset_path, "image", "*.bmp")))
    masks = sorted(glob(os.path.join(dataset_path, "mask", "*.bmp")))
    #print(type(images))
    test_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=test_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=test_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [ ]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x                                ## (256, 256, 3)

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)                    ## (256, 256)
    x = np.expand_dims(x, axis=-1)              ## (256, 256, 1)
    return x

In [ ]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

In [ ]:
""" Folder For saving result"""
create_dir("Files")

In [ ]:
# seeding 

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
""" Hyperparameters """
batch_size = 16
lr = 1e-4 ## (0.0001)
num_epoch = 50

In [ ]:
""" Dataset : 60/20/20 """
dataset_path = "/content/augDataset"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)


In [ ]:
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")

Train: 480 - 480
Valid: 160 - 160
Test: 160 - 160


In [ ]:
train_dataset = tf_dataset(train_x, train_y, batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size)
test_dataset = tf_dataset(test_x, test_y, batch_size)

In [ ]:
train_steps = len(train_x)//batch_size
valid_steps = len(valid_x)//batch_size
test_steps = len(test_x)//batch_size

In [ ]:

if len(train_x) % batch_size != 0:
  train_steps += 1

if len(valid_x) % batch_size != 0:
  valid_steps += 1

if len(test_x) % batch_size != 0:
  test_steps += 1

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
inputs = tf.keras.layers.Input((256, 256, 3))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)


#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)


#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

In [ ]:

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

metrics = [dice_coef, iou, Recall(), Precision()]
model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 lambda (Lambda)                (None, 256, 256, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv2d_38 (Conv2D)             (None, 256, 256, 16  448         ['lambda[0][0]']                 
                                )                                                                 
                                                                                            

In [ ]:
model_path="/content/Files/model.h5"
csv_path="/content/Files/data.csv"
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
]

In [ ]:
model.fit(
        train_dataset,
        epochs=num_epoch,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Epoch 1/50
30/30 [==============================] - ETA: 0s - loss: 0.6344 - dice_coef: 0.3335 - iou: 0.2005 - recall_3: 5.8500e-05 - precision_3: 0.1584
Epoch 1: val_loss improved from inf to 0.64015, saving model to /content/Files/model.h5
30/30 [==============================] - 8s 195ms/step - loss: 0.6344 - dice_coef: 0.3335 - iou: 0.2005 - recall_3: 5.8500e-05 - precision_3: 0.1584 - val_loss: 0.6402 - val_dice_coef: 0.3666 - val_iou: 0.2248 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/50
30/30 [==============================] - ETA: 0s - loss: 0.5833 - dice_coef: 0.3435 - iou: 0.2079 - recall_3: 1.8807e-06 - precision_3: 0.0486
Epoch 2: val_loss improved from 0.64015 to 0.56583, saving model to /content/Files/model.h5
30/30 [==============================] - 5s 177ms/step - loss: 0.5833 - dice_coef: 0.3435 - iou: 0.2079 - recall_3: 1.8807e-06 - precision_3: 0.0486 - val_loss: 0.5658 - val_dice_coef: 0.3874 - val_iou: 0.2405 - val_recall_3: 0.

In [ ]:
model.save('Unet.hdf5')

In [ ]:
!mkdir results
from tqdm import tqdm

In [ ]:
def read_test_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)
    return ori_x, x                                ## (1, 256, 256, 3)


def read_test_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)                    ## (256, 256)
    return ori_x, x

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)
model.load_weights('/content/Unet.hdf5')

In [ ]:
res=model.evaluate(train_dataset, steps=test_steps)
print('\n-------------On Train  Set--------------------------\n')
print('________________________')
print('Dice Coef:      |   {:.2f}  |'.format(res[1]*100))
print('IoU:            |   {:.2f}  |'.format(res[2]*100))
print('Recall:         |   {:.2f}  |'.format(res[3]*100))
print('Precision:      |   {:.2f}  |'.format(res[4]*100))
print('________________________')
res=model.evaluate(test_dataset, steps=test_steps)
print('\n-------------On Test  Set--------------------------\n')
print('________________________')
print('Dice Coef:      |   {:.2f}  |'.format(res[1]*100))
print('IoU:            |   {:.2f}  |'.format(res[2]*100))
print('Recall:         |   {:.2f}  |'.format(res[3]*100))
print('Precision:      |   {:.2f}  |'.format(res[4]*100))
print('________________________')
res=model.evaluate(valid_dataset, steps=test_steps)
print('\n-------------On valid  Set--------------------------\n')
print('________________________')
print('Dice Coef:      |   {:.2f}  |'.format(res[1]*100))
print('IoU:            |   {:.2f}  |'.format(res[2]*100))
print('Recall:         |   {:.2f}  |'.format(res[3]*100))
print('Precision:      |   {:.2f}  |'.format(res[4]*100))
print('________________________')

10/10 [==============================] - 1s 59ms/step - loss: 0.1664 - dice_coef: 0.8478 - iou: 0.7362 - recall_3: 0.8520 - precision_3: 0.9377

-------------On Train  Set--------------------------

________________________
Dice Coef:      |   84.78  |
IoU:            |   73.62  |
Recall:         |   85.20  |
Precision:      |   93.77  |
________________________
10/10 [==============================] - 1s 49ms/step - loss: 0.1912 - dice_coef: 0.8352 - iou: 0.7186 - recall_3: 0.8399 - precision_3: 0.9276

-------------On Test  Set--------------------------

________________________
Dice Coef:      |   83.52  |
IoU:            |   71.86  |
Recall:         |   83.99  |
Precision:      |   92.76  |
________________________
10/10 [==============================] - 1s 50ms/step - loss: 0.1670 - dice_coef: 0.8474 - iou: 0.7360 - recall_3: 0.8513 - precision_3: 0.9437

-------------On valid  Set--------------------------

________________________
Dice Coef:      |   84.74  |
IoU:            | 